<a href="https://colab.research.google.com/github/nujudaly/T5/blob/main/Copy_of_RAG_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [29]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/usr/local/lib/python3.10/d

In [3]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

## Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [373]:
import pandas as pd
df = pd.read_csv("/content/Dataset.csv")
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [374]:
len(df['المخالفة'].max())

39

In [375]:
df['المخالفة'][i]

'المجازفة بعبور الأودية والشعاب أثناء جريانها.'

In [376]:
max='a'
for i in range(103):
  if(df['المخالفة'][i])>max:
    max=df['المخالفة'][i]
max

'وقوف المركبة في أماكن غير مخصصة للوقوف.'

In [377]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [378]:
import markdown
import os

In [395]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [396]:
for i in range(len(df)):

    title = df['الغرامة'].iloc[i]
    content = df['المخالفة'].iloc[i]

    markdown_content = f"# {title}\n\n"
    markdown_content += f"{content}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)


In [397]:
markdown_texts = []

for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)


## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [398]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings


In [399]:
len("عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية")

99

In [400]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=120, chunk_overlap=10)
documents = text_splitter.create_documents(markdown_texts)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [401]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template greets the user first, states that the information provided could be incorrect, and advises the user to visit the traffic initiative website to verify. Additionally, provide the user with advice in Arabic, ensuring it stays within the given context.

In [402]:
import json
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq


In [403]:

#  Ensure the template greets the user first




PROMPT_TEMPLATE="""
:مرحبا بك, بامكانك سؤالي عن اي مخالفة،
 للمزيد يرجى التحقق الرجاء زيارة الموقع الرسمي
Context: {context}
Question: {question}
:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)


MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)





## Step 7: Initialize the Language Model


Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [404]:
groq_api_key = "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"
llm =  ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama-3.1-8b-instant")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [405]:




MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [408]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=5)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response





## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [411]:


response = query_rag("ماهي غرامة عدم الوقوف وقوفاً تاماً عند إشارة")
response




Prompt after formatting:

:مرحبا بك, بامكانك سؤالي عن اي مخالفة،
 للمزيد يرجى التحقق الرجاء زيارة الموقع الرسمي
Context: <p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له

<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له

<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له

<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له

<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له
Question: ماهي غرامة عدم الوقوف وقوفاً تاماً عند إشارة
:


> Finished chain.


{'context': '<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له\n\n<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له\n\n<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له\n\n<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له\n\n<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له',
 'question': 'ماهي غرامة عدم الوقوف وقوفاً تاماً عند إشارة',
 'text': 'الجواب على سؤالك هو: \nغرامة عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له هي 300 درهم.'}